# Source localization pipeline - Stage2
- coregistration
- source localization
- report generation
 
 


#### Author
Rudi Kreidenhuber <Rudi.Kreidenhuber@gmail.com>
#### License
BSD (3-clause)

# Coregistration

In [1]:
from src import SubjectDropDowner
import os
import glob
import mne

# configuration
input_dir = "/home/meg/Schreibtisch/new_patients/processed"
bids_root = "/run/media/meg/DATA/MEG/BIDS_clinic/derivatives"
n_jobs = openmp = 30
extras_directory = os.path.join(input_dir, "extras")
FS_SUBJECTS_DIR = "/usr/local/freesurfer/subjects/"
spacing = "ico4"

print("Configuration:")
for dir in [input_dir, bids_root, extras_directory, FS_SUBJECTS_DIR]:
    print(f"{dir}")


def get_subject_list(input_dir = input_dir) -> list:
    fiflist = [os.path.basename(f) for f in glob.glob(os.path.join(input_dir, "*.fif"))]
    fiflist = [f.split("_")[0] for f in fiflist]
    print(fiflist)
    return fiflist

subjectlist = get_subject_list()

dl = SubjectDropDowner.SubjectDropDowner(subjectlist)
drop_menu = dl.create_subject_dropdown_widget()
print(f"\n\nChoose patient/ subject to process further:")
drop_menu

Configuration:
/home/meg/Schreibtisch/new_patients/processed
/run/media/meg/DATA/MEG/BIDS_clinic/derivatives
/home/meg/Schreibtisch/new_patients/processed/extras
/usr/local/freesurfer/subjects/
['sub-AO12091991', 'sub-PA27111988', 'sub-PG05071961', 'sub-VA09051993']


Choose patient/ subject to process further:


In [2]:
subject = dl.get_subject_dropdown().value

if not subject.startswith("sub-"):
    subject = "sub-" + subject
    
print(f"Chosen Subject: {subject}")



Chosen Subject: sub-VA09051993


In [4]:
from src import Folderer, Anatomist

# create output folders
dfc = Folderer.DerivativesFoldersCreator(BIDS_root=bids_root, 
                                            extras_directory=extras_directory, 
                                            subject=subject)
dfc.make_derivatives_folders()

# copy freesurfer files to local subjects_dir
try:
    segmentation = os.path.join(os.path.abspath(FS_SUBJECTS_DIR), subject)
    target = os.path.join(dfc.fanat, subject)
    if not os.path.isdir(target):
        os.mkdir(target)
    dfc._recursive_overwrite(segmentation, target)
except Exception as e:
    print(f"Couldn't copy freesurfer segmentation\n--> {e}.")
    
# create source models
#sourcerer = Anatomist.SourceModeler(subjects_dir=dfc.fanat, subject=subject, spacing=spacing, n_jobs=n_jobs)
#sourcerer.calculate_source_models()


transfile = os.path.join(dfc.ftrans, subject + "-trans.fif")
print("Starting coregistration...")
if os.path.isfile(transfile):
    print(f"Skipping coregistration, because a transfile ({transfile}) already exists")
else:
    print(f"\n\n\n--> Transfile should be called: {transfile}\n\n\n")
    try:
        mne.gui.coregistration(subject=subject, subjects_dir=dfc.fanat, inst=bids_root, advanced_rendering=False) # BIDS: inst=raw.filenames[0])
    except:
        print("failed with bids_derivatives folder")
        filename = subject + "_prepped.fif"
        rawfile = os.path.join(input_dir, "processed", filename)
        print(f"Coregistration - rawfile used was: {rawfile}")
        mne.gui.coregistration(subject=subject, subjects_dir=dfc.fanat, inst=rawfile, advanced_rendering=False)

Starting coregistration...



--> Transfile should be called: /run/media/meg/DATA/MEG/BIDS_clinic/derivatives/derivatives/sub-VA09051993/trans_files/sub-VA09051993-trans.fif



    Triangle neighbors and vertex normals...
Using low resolution head model in /run/media/meg/DATA/MEG/BIDS_clinic/derivatives/derivatives/sub-VA09051993/freesurfer/sub-VA09051993/bem/outer_skin.surf
    Triangle neighbors and vertex normals...
Estimating fiducials from fsaverage.
    Triangle neighbors and vertex normals...
Using low resolution head model in /run/media/meg/DATA/MEG/BIDS_clinic/derivatives/derivatives/sub-VA09051993/freesurfer/sub-VA09051993/bem/outer_skin.surf
    Triangle neighbors and vertex normals...
Estimating fiducials from fsaverage.
Estimating fiducials from fsaverage.


/tmp/ipykernel_392106/4038135225.py:31: RuntimeWarning: The parameter advanced_rendering is not supported with the pyvistaqt 3d backend. It will be ignored.
  mne.gui.coregistration(subject=subject, subjects_dir=dfc.fanat, inst=bids_root, advanced_rendering=False) # BIDS: inst=raw.filenames[0])


Placing MRI fiducials - LPA
Using outer_skin.surf for head surface.
Placing MRI fiducials - LPA
Using outer_skin.surf for head surface.
Channel types::	eeg: 64
Getting helmet for system 306m
Aligning using fiducials
Start median distance:  12.99 mm
End   median distance:   4.69 mm
Fitting fiducials finished in 0.01 seconds.
Channel types::	eeg: 64
Aligning using ICP
Start     median distance:   4.69 mm
  ICP  1  median distance:   3.78 mm
Fitting ICP - iteration 1
Channel types::	eeg: 64
  ICP  2  median distance:   3.27 mm
Fitting ICP - iteration 2
Channel types::	eeg: 64
  ICP  3  median distance:   2.99 mm
Fitting ICP - iteration 3
Channel types::	eeg: 64
  ICP  4  median distance:   2.88 mm
Fitting ICP - iteration 4
Channel types::	eeg: 64
  ICP  5  median distance:   2.84 mm
Fitting ICP - iteration 5
Channel types::	eeg: 64
End       median distance:   2.84 mm
Fitting ICP finished in 1.59 seconds and 5 iterations.
Channel types::	eeg: 64
Channel types::	eeg: 64
Channel types::	eeg